<a href="https://colab.research.google.com/github/digichriz/EMBA2026-Lab/blob/main/exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My awesome LLM-Application

###Setup
First, make the necessary imports.
Note that further imports may have to be made in addition to the ones below, if your application uses additional fetures such as loaders and tools. You can find the code for these imports in the respective sections of the tutorial notebooks.

In [19]:
!pip install -q langchain langchain-community langchain-core langchain-openai langchain-huggingface langchain-google-genai
from google.colab import userdata
#from langchain.chains import SequentialChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import os
import pprint
import getpass
from IPython.display import Markdown
import IPython.display as ipd
from PIL import Image
import urllib.request

Then, assign the API keys to be able to use OpenAI, Google Serper, Huggingface, etc.

When working with sensitive information like API keys or passwords in Google Colab, it's crucial to handle data securely. As you learnt in the tutorial session, two common approaches for this are using **Colab's Secrets Manager**, which stores and retrieves secrets without exposing them in the notebook, and `getpass`, a Python function that securely prompts users to input secrets during runtime without showing them. Both methods help ensure your sensitive data remains protected.

In [37]:
# **********************************************************************************************
# *                                                                                            *
# *   ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  *
# *                                                                                            *
# *                                      R E P U T A T I O   A I                                 *
# *                                                                                            *
# *                     Multi-AI perception analysis enriched with public web data              *
# *                                                                                            *
# *                                                                                            *
# *   What this notebook does                                                                   *
# *   ----------------------------------------------------------------------------------------  *
# *   • Builds a fact sheet from publicly available online information                          *
# *   • Analyzes how different AI models perceive the same entity                               *
# *   • Create an image (diagram) out of the analysis                                           *
# *   • Separates verifiable facts from AI-generated sentiment and interpretation               *
# *                                                                                            *
# *                                                                                            *
# *   Built & maintained by                                                                     *
# *   ----------------------------------------------------------------------------------------  *
# *   Christian Schmid                                                                          *
# *                                                                                            *
# *                                                                                            *
# *                                                                                            *
# *   ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★  ★ *
# *                                                                                            *
# **********************************************************************************************



#API Key Imports
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")

#When using Colab Secret Manager
#os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')


#global vars
search = GoogleSerperAPIWrapper()
llmOpenAI = ChatOpenAI(model="gpt-5-mini")
llmGemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

#I use openrouter API to get free access on a deepseek Model as 3. LLM
llmDeepSeek = ChatOpenAI(
    model="nex-agi/deepseek-v3.1-nex-n1:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
)

####User Input
The user is expected to provide a company name via the chat interface. Since this is not available here, I’ve stored an example companiy in the variable. Feel free to modify it as needed for your evaluation for more examples

In [24]:
user_input = "Georg Fischer AG"

###Prompt Templates

In [38]:

#Template to feed the different AI models
report_prompt_template = PromptTemplate(
    input_variables=["company"],
    template = ("""Based on the company name provided: {company}, generate a concise company profile and assessment.
                Include:
                - A brief description of what the company does
                - Its primary industry and general market positioning
                - Perceived strengths and weaknesses
                - Overall reputation and maturity level

                Rules:
                - Use only widely known, high-level information
                - Do not speculate or invent facts
                - Explicitly state uncertainty where information is unclear or unavailable
                - Use a neutral, analytical tone
                - Maximum length: 150–200 words"""
               )
)

#Template for one of the AI (in my case openAI) to analyze the sentiment of the results from the AI models
sentiment_prompt_template = PromptTemplate(
    input_variables=["company_analysis_sentiment"],
    template = ("""You are analyzing the output of an AI-generated company assessment.
                Input text:
                {company_analysis_sentiment}

                Your task:
                - Determine whether the overall sentiment of the text is predominantly positive, neutral, or negative
                - Explain briefly why this sentiment classification applies, referring only to the content of the text
                - Provide a sentiment rating on a scale from 1 to 5, where:
                  1 = very negative
                  2 = mostly negative
                  3 = neutral or balanced
                  4 = mostly positive
                  5 = very positive

                Rules:
                - Do not introduce new information or external knowledge
                - Base your evaluation strictly on tone, wording, and emphasis in the provided text
                - If the sentiment is mixed, explain which aspects pull it in each direction
                - Keep the total response under 120 words
                - Use a neutral, analytical tone"""
               )
)

###Full Chain / Workflow Execution

In [40]:
#Function to invoke the different LLMs to gather info about the company

def analyze_company(llm, user_input):
    prompt = report_prompt_template.format(company=user_input)
    response = llm.invoke(prompt)

    content = response.content if hasattr(response, "content") else str(response)
    display(Markdown(content))
    return content


company_text = analyze_company(
    llm=llmDeepSeek,
    user_input=user_input
)


**Georg Fischer AG – Company Profile and Assessment**

Based on publicly available information, Georg Fischer AG (GF) is a Swiss industrial manufacturing group with operations spanning three core divisions: GF Piping Systems, GF Casting Solutions, and GF Machining Solutions. The company specializes in piping systems for water and gas transport, high-pressure die-casting solutions for automotive applications, and precision machining equipment. GF is positioned as a mid-sized global industrial supplier, with a focus on highly engineered components and systems that serve the construction, automotive, aerospace, and industrial manufacturing sectors.

**Strengths & Weaknesses:** Key strengths include a well-diversified portfolio, a strong reputation for engineering quality, and entrenched relationships in regulated industries such as utilities and automotive. GF Piping Systems enjoys high brand recognition in water infrastructure. Weaknesses may include limited exposure to rapidly growing sectors beyond its core markets and sensitivity to industrial downturns. Some product lines face intense competition from larger industrial conglomerates.

**Reputation & Maturity:** GF is considered a mature, stable industrial manufacturer with a long operating history dating back to the 19th century. While not as widely known as major multi-industry conglomerates, it maintains a solid reputation for reliability and technical competence within its niche markets.